Script to propagate LG modes over a certain distance, whilst subject to particulate absorbers in the beam path.
All units are in metres.

In [18]:
import propagation_functions as prop
import particle_functions as parts
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, fixed

Particles per second: 0.048246
Seconds per particle: 20.727106910417444s
Time to pass: 4.2415899999999995ms


Setting initial parameters.
Care should be taken to ensure beam does not get too close to the edge: phase can wrap and cause funky behaviour!

In [19]:
res = 512 #i wouldn't recommend going too much higher than this for resolution
screen_width = 4 # the cross sectional width and height of the simulation
wvl = 0.65e-9 
distance = 1
n = 10

Wrapper function containing beam definition, launch, propagation, and interaction with particulates.
Reason for this wrapper is so we can run it interactively below.

In [29]:
def do_it_all(L=1, distance=1000, maalox=2, l0=0.001, L0=100.0, r0=0.1, turb=False):
    # generate instance of beam class
    beam_waist = 0.5
    beam = prop.BeamProfile(512, beam_waist*16, 6.5e-9)

    #generate the beam specifics, there is also a hg option, but with these options it is a simple collimated gaussian beam 
    
    initial_z = 0
    beam.laguerre_gaussian_beam(L, 0, beam_waist, initial_z)

    #propagate the beam some distance 
    beam.free_space_prop(distance)

    n, _ = parts.total_particles(maalox, 60)
    n = int(n)

    absorbers = prop.AbsorberScreen(4, 512, 11e-6, n)
    absorbers.generate_absorbers()
    beam.apply_absorber_screen(absorbers.grid)

    #generate the phase screen - don't worry too much
    if turb:
        phz = prop.PhaseScreen(screen_width, res, r0, l0, L0)
        phz.mvk_screen()
        phz.mvk_sh_screen()

        # #APPLY THE PHASE SCREEN TO BEAM 
        beam.apply_phase_screen(phz.phz + phz.phz_lo)

    beam.free_space_prop(distance)

    fig, (ax0, ax1) = plt.subplots(1, 2)
    ax0.imshow(np.abs(beam.field))
    ax1.imshow(np.angle(beam.field), cmap = 'hsv')
    plt.show()

In [33]:
w = interactive(do_it_all, L=(0,5), distance=(0,10000), maalox=(0,10,0.1), l0=(0, 0.1, 0.001), L0=(1,500), r0=(0.001,1,0.01))
w

interactive(children=(IntSlider(value=1, description='L', max=5), IntSlider(value=1000, description='distance'…